In [40]:
# Import relevent libraries 
import os
import psycopg2
import pandas as pd
from psycopg2 import Error

# Read in the data file that we need to insert
df = pd.read_excel("/Users/tahir/Desktop/Github/documents/roots/Data/Cleaned_Root_letters.xlsx")

In [41]:
df.head()

,ID,ARABIC,Transliteration,Root_Letters
0,1:1,بِسْمِ,bis'mi,س م و
1,1:1,اللَّهِ,l-lahi,ا ل ه
2,1:1,الرَّحْمَنِ,l-raḥmāni,ر ح م
3,1:1,الرَّحِيمِ,l-raḥīmi,ر ح م
4,1:2,الْحَمْدُ,al-ḥamdu,ح م د


In [42]:
# Create another column for surah and ayat, extracted from ID
surah, ayah = [], []
for row in df["ID"]:
    surah.append( row.split(":")[0] ) 
    ayah.append( row.split(":")[1] )

df["Surah"] = surah
df["Ayah"] = ayah

In [43]:
df

,ID,ARABIC,Transliteration,Root_Letters,Surah,Ayah
0,1:1,بِسْمِ,bis'mi,س م و,1,1
1,1:1,اللَّهِ,l-lahi,ا ل ه,1,1
2,1:1,الرَّحْمَنِ,l-raḥmāni,ر ح م,1,1
3,1:1,الرَّحِيمِ,l-raḥīmi,ر ح م,1,1
4,1:2,الْحَمْدُ,al-ḥamdu,ح م د,1,2
...,...,...,...,...,...,...
77426,114:5,صُدُورِ,ṣudūri,ص د ر,114,5
77427,114:5,النَّاسِ,l-nāsi,ن و س,114,5
77428,114:6,مِنَ,mina,م ن,114,6
77429,114:6,الْجِنَّةِ,l-jinati,ج ن ن,114,6


In [3]:
# Connect to the Postgres SQL Database 
try:         
    # Define DB connection parameters 
    dbHost = '127.0.0.1'
    dbPort = 5434
    dbUser = 'qj'
    dbPassword= 'Yatathakar123!'
    dbName = 'quranJourney'

    # connect to the PostgreSQL database
    connection = psycopg2.connect(user=dbUser,
                              password=dbPassword,
                              host=dbHost,
                              port=dbPort,
                              database=dbName)
    cursor = connection.cursor()

except (Exception, Error) as error:
    print("Error while connecting to PostgreSQL", error)

In [24]:
postgreSQL_select_Query = "select * from quran_text"

cursor.execute(postgreSQL_select_Query)
print("Selecting rows from quran table using cursor.fetchall")
quran_text_full = cursor.fetchall()

print("Print each row and it's columns values")
index, surah, ayah, text = [], [], [], [] 
for row in quran_text_full:
    index.append(row[0])
    surah.append(row[1])
    ayah.append(row[2])
    text.append(row[3])
    
quran_text_df = pd.DataFrame(
    {'Index': index,
     'Surah': surah,
     'Ayah': ayah,
     'Text': text
    })
quran_text_df.head(10)

Selecting rows from quran table using cursor.fetchall
Print each row and it's columns values


,Index,Surah,Ayah,Text
0,1,1,1,بِسْمِ اللَّهِ الرَّحْمَـٰنِ الرَّحِيمِ
1,2,1,2,الْحَمْدُ لِلَّهِ رَبِّ الْعَالَمِينَ
2,3,1,3,الرَّحْمَـٰنِ الرَّحِيمِ
3,4,1,4,مَالِكِ يَوْمِ الدِّينِ
4,5,1,5,إِيَّاكَ نَعْبُدُ وَإِيَّاكَ نَسْتَعِينُ
5,6,1,6,اهْدِنَا الصِّرَاطَ الْمُسْتَقِيمَ
6,7,1,7,صِرَاطَ الَّذِينَ أَنْعَمْتَ عَلَيْهِمْ غَيْرِ...
7,8,2,1,بِسْمِ اللَّهِ الرَّحْمَـٰنِ الرَّحِيمِ الم
8,9,2,2,ذَٰلِكَ الْكِتَابُ لَا رَيْبَ ۛ فِيهِ ۛ هُدًى ...
9,10,2,3,الَّذِينَ يُؤْمِنُونَ بِالْغَيْبِ وَيُقِيمُونَ...


In [27]:
# Close the DB connection 
cursor.close()
connection.close()
print("PostgreSQL connection is closed")

PostgreSQL connection is closed


# Now that we have the data... its time to prep it for our db

In [70]:
df

,ID,ARABIC,Transliteration,Root_Letters,Surah,Ayah
0,1:1,بِسْمِ,bis'mi,س م و,1,1
1,1:1,اللَّهِ,l-lahi,ا ل ه,1,1
2,1:1,الرَّحْمَنِ,l-raḥmāni,ر ح م,1,1
3,1:1,الرَّحِيمِ,l-raḥīmi,ر ح م,1,1
4,1:2,الْحَمْدُ,al-ḥamdu,ح م د,1,2
...,...,...,...,...,...,...
77426,114:5,صُدُورِ,ṣudūri,ص د ر,114,5
77427,114:5,النَّاسِ,l-nāsi,ن و س,114,5
77428,114:6,مِنَ,mina,م ن,114,6
77429,114:6,الْجِنَّةِ,l-jinati,ج ن ن,114,6


In [67]:
df["Surah"] = df["Surah"].values.astype(int)
df["Ayah"] = df["Ayah"].values.astype(int)

In [68]:
df.dtypes

ID                 object
ARABIC             object
Transliteration    object
Root_Letters       object
Surah               int64
Ayah                int64
dtype: object

In [69]:
quran_text_df

,Index,Surah,Ayah,Text
0,1,1,1,بِسْمِ اللَّهِ الرَّحْمَـٰنِ الرَّحِيمِ
1,2,1,2,الْحَمْدُ لِلَّهِ رَبِّ الْعَالَمِينَ
2,3,1,3,الرَّحْمَـٰنِ الرَّحِيمِ
3,4,1,4,مَالِكِ يَوْمِ الدِّينِ
4,5,1,5,إِيَّاكَ نَعْبُدُ وَإِيَّاكَ نَسْتَعِينُ
...,...,...,...,...
6231,6232,114,2,مَلِكِ النَّاسِ
6232,6233,114,3,إِلَـٰهِ النَّاسِ
6233,6234,114,4,مِن شَرِّ الْوَسْوَاسِ الْخَنَّاسِ
6234,6235,114,5,الَّذِي يُوَسْوِسُ فِي صُدُورِ النَّاسِ


In [71]:
df3  = quran_text_df.merge(df, on = ["Surah", "Ayah" ], how = 'inner')

In [72]:
df3

,Index,Surah,Ayah,Text,ID,ARABIC,Transliteration,Root_Letters
0,1,1,1,بِسْمِ اللَّهِ الرَّحْمَـٰنِ الرَّحِيمِ,1:1,بِسْمِ,bis'mi,س م و
1,1,1,1,بِسْمِ اللَّهِ الرَّحْمَـٰنِ الرَّحِيمِ,1:1,اللَّهِ,l-lahi,ا ل ه
2,1,1,1,بِسْمِ اللَّهِ الرَّحْمَـٰنِ الرَّحِيمِ,1:1,الرَّحْمَنِ,l-raḥmāni,ر ح م
3,1,1,1,بِسْمِ اللَّهِ الرَّحْمَـٰنِ الرَّحِيمِ,1:1,الرَّحِيمِ,l-raḥīmi,ر ح م
4,2,1,2,الْحَمْدُ لِلَّهِ رَبِّ الْعَالَمِينَ,1:2,الْحَمْدُ,al-ḥamdu,ح م د
...,...,...,...,...,...,...,...,...
77426,6235,114,5,الَّذِي يُوَسْوِسُ فِي صُدُورِ النَّاسِ,114:5,صُدُورِ,ṣudūri,ص د ر
77427,6235,114,5,الَّذِي يُوَسْوِسُ فِي صُدُورِ النَّاسِ,114:5,النَّاسِ,l-nāsi,ن و س
77428,6236,114,6,مِنَ الْجِنَّةِ وَالنَّاسِ,114:6,مِنَ,mina,م ن
77429,6236,114,6,مِنَ الْجِنَّةِ وَالنَّاسِ,114:6,الْجِنَّةِ,l-jinati,ج ن ن


In [73]:
df4 = df3.merge(arabic_words_df, on = "ARABIC", how = 'inner')
df4

,Index,Surah,Ayah,Text,ID,ARABIC,Transliteration,Root_Letters,primary_key,RootID
0,1,1,1,بِسْمِ اللَّهِ الرَّحْمَـٰنِ الرَّحِيمِ,1:1,بِسْمِ,bis'mi,س م و,2000,890
1,1514,11,41,وَقَالَ ارْكَبُوا فِيهَا بِسْمِ اللَّهِ مَجْرَ...,11:41,بِسْمِ,bis'mi,س م و,2000,890
2,3189,27,30,إِنَّهُ مِن سُلَيْمَانَ وَإِنَّهُ بِسْمِ اللَّ...,27:30,بِسْمِ,bis'mi,س م و,2000,890
3,1,1,1,بِسْمِ اللَّهِ الرَّحْمَـٰنِ الرَّحِيمِ,1:1,اللَّهِ,l-lahi,ا ل ه,2001,114
4,30,2,23,وَإِن كُنتُمْ فِي رَيْبٍ مِّمَّا نَزَّلْنَا عَ...,2:23,اللَّهِ,l-lahi,ا ل ه,2001,114
...,...,...,...,...,...,...,...,...,...,...
77786,6230,113,5,وَمِن شَرِّ حَاسِدٍ إِذَا حَسَدَ,113:5,حَسَدَ,ḥasada,ح س د,19618,869
77787,6232,114,2,مَلِكِ النَّاسِ,114:2,مَلِكِ,maliki,م ل ك,19619,1178
77788,6234,114,4,مِن شَرِّ الْوَسْوَاسِ الْخَنَّاسِ,114:4,الْوَسْوَاسِ,l-waswāsi,و س و س,19620,1606
77789,6234,114,4,مِن شَرِّ الْوَسْوَاسِ الْخَنَّاسِ,114:4,الْخَنَّاسِ,l-khanāsi,خ ن س,19621,217


In [76]:
df4.head(30)

,Index,Surah,Ayah,Text,ID,ARABIC,Transliteration,Root_Letters,primary_key,RootID
0,1,1,1,بِسْمِ اللَّهِ الرَّحْمَـٰنِ الرَّحِيمِ,1:1,بِسْمِ,bis'mi,س م و,2000,890
1,1514,11,41,وَقَالَ ارْكَبُوا فِيهَا بِسْمِ اللَّهِ مَجْرَ...,11:41,بِسْمِ,bis'mi,س م و,2000,890
2,3189,27,30,إِنَّهُ مِن سُلَيْمَانَ وَإِنَّهُ بِسْمِ اللَّ...,27:30,بِسْمِ,bis'mi,س م و,2000,890
3,1,1,1,بِسْمِ اللَّهِ الرَّحْمَـٰنِ الرَّحِيمِ,1:1,اللَّهِ,l-lahi,ا ل ه,2001,114
4,30,2,23,وَإِن كُنتُمْ فِي رَيْبٍ مِّمَّا نَزَّلْنَا عَ...,2:23,اللَّهِ,l-lahi,ا ل ه,2001,114
5,34,2,27,الَّذِينَ يَنقُضُونَ عَهْدَ اللَّهِ مِن بَعْدِ...,2:27,اللَّهِ,l-lahi,ا ل ه,2001,114
6,67,2,60,وَإِذِ اسْتَسْقَىٰ مُوسَىٰ لِقَوْمِهِ فَقُلْنَ...,2:60,اللَّهِ,l-lahi,ا ل ه,2001,114
7,68,2,61,وَإِذْ قُلْتُمْ يَا مُوسَىٰ لَن نَّصْبِرَ عَلَ...,2:61,اللَّهِ,l-lahi,ا ل ه,2001,114
8,68,2,61,وَإِذْ قُلْتُمْ يَا مُوسَىٰ لَن نَّصْبِرَ عَلَ...,2:61,اللَّهِ,l-lahi,ا ل ه,2001,114
9,71,2,64,ثُمَّ تَوَلَّيْتُم مِّن بَعْدِ ذَٰلِكَ ۖ فَلَو...,2:64,اللَّهِ,l-lahi,ا ل ه,2001,114


In [84]:
df5 = df4.drop_duplicates()
df5.nunique()

Index               6236
Surah                114
Ayah                 286
Text                6060
ID                  6236
ARABIC             17613
Transliteration    17608
Root_Letters        1799
primary_key        17623
RootID              1799
dtype: int64

In [92]:
df5["index2"] = df5["Index"]
df5 = df5.drop(["Index"], axis=1)

/var/folders/hc/rv0ch5hn293cmmlwhw7w_drh0000gn/T/ipykernel_7525/513230290.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df5["index2"] = df5["Index"]


In [95]:
df5.index

Int64Index([    0,     1,     2,     3,     4,     5,     6,     7,     9,
               10,
            ...
            77781, 77782, 77783, 77784, 77785, 77786, 77787, 77788, 77789,
            77790],
           dtype='int64', length=74519)

In [97]:
# Prep data
# Put the data in the format the db accepts
text_to_word_data = []
for row in df5.index:
    AyahID = text_to_word_data.loc[row,"index2"]
    WordID = text_to_word_data.loc[row,"primary_key"]
    
    text_to_word_data.append((AyahID, WordID))


AttributeError: 'list' object has no attribute 'loc'

In [58]:
df4 = df3.merge(arabic_words_df, left_on ="ARABIC", right_on = "ARABIC", how = 'outer')
df4.head()

,Index,Surah,Ayah_x,Text,ID,ARABIC,Transliteration,Root_Letters,Ayah_y,primary_key,RootID
0,1,1,1,بِسْمِ اللَّهِ الرَّحْمَـٰنِ الرَّحِيمِ,1:1,بِسْمِ,bis'mi,س م و,1,2000,890
1,2,1,2,الْحَمْدُ لِلَّهِ رَبِّ الْعَالَمِينَ,1:1,بِسْمِ,bis'mi,س م و,1,2000,890
2,3,1,3,الرَّحْمَـٰنِ الرَّحِيمِ,1:1,بِسْمِ,bis'mi,س م و,1,2000,890
3,4,1,4,مَالِكِ يَوْمِ الدِّينِ,1:1,بِسْمِ,bis'mi,س م و,1,2000,890
4,5,1,5,إِيَّاكَ نَعْبُدُ وَإِيَّاكَ نَسْتَعِينُ,1:1,بِسْمِ,bis'mi,س م و,1,2000,890


In [61]:
df5 = df4.drop_duplicates()

In [63]:
df5

,Index,Surah,Ayah_x,Text,ID,ARABIC,Transliteration,Root_Letters,Ayah_y,primary_key,RootID
0,1,1,1,بِسْمِ اللَّهِ الرَّحْمَـٰنِ الرَّحِيمِ,1:1,بِسْمِ,bis'mi,س م و,1,2000,890
1,2,1,2,الْحَمْدُ لِلَّهِ رَبِّ الْعَالَمِينَ,1:1,بِسْمِ,bis'mi,س م و,1,2000,890
2,3,1,3,الرَّحْمَـٰنِ الرَّحِيمِ,1:1,بِسْمِ,bis'mi,س م و,1,2000,890
3,4,1,4,مَالِكِ يَوْمِ الدِّينِ,1:1,بِسْمِ,bis'mi,س م و,1,2000,890
4,5,1,5,إِيَّاكَ نَعْبُدُ وَإِيَّاكَ نَسْتَعِينُ,1:1,بِسْمِ,bis'mi,س م و,1,2000,890
...,...,...,...,...,...,...,...,...,...,...,...
9199417,6232,114,2,مَلِكِ النَّاسِ,114:5,يُوَسْوِسُ,yuwaswisu,و س و س,5,19622,1606
9199418,6233,114,3,إِلَـٰهِ النَّاسِ,114:5,يُوَسْوِسُ,yuwaswisu,و س و س,5,19622,1606
9199419,6234,114,4,مِن شَرِّ الْوَسْوَاسِ الْخَنَّاسِ,114:5,يُوَسْوِسُ,yuwaswisu,و س و س,5,19622,1606
9199420,6235,114,5,الَّذِي يُوَسْوِسُ فِي صُدُورِ النَّاسِ,114:5,يُوَسْوِسُ,yuwaswisu,و س و س,5,19622,1606


In [44]:
arabic_words_df

,ARABIC,primary_key,RootID
0,بِسْمِ,2000,890
1,السَّمَاءِ,2158,890
2,وَالسَّمَاءَ,2199,890
3,سَمَاوَاتٍ,2302,890
4,الْأَسْمَاءَ,2325,890
...,...,...,...
17618,مَسَدٍ,19606,1727
17619,الصَّمَدُ,19607,118
17620,كُفُوًا,19610,1641
17621,وَقَبَ,19614,1172


In [49]:
AyahID = quran_text_df["Index"]
WordID = 

In [33]:
# Prep Data for RootWords Table 
lst = list(df['Root_Letters'])
lst = set(lst)

# Make it into a dataframe, and give it an index / ID column 
df_roots = pd.DataFrame(lst, columns=['Root_Words'])
df_roots = df_roots.reset_index()

# df_roots["Root_Words"] = lst
df_roots["Root_Words"] = df_roots["Root_Words"].values.astype(str)

# Put the data in the format the db accepts
root_data = []
for row in df_roots.index:
    arabic_text = df_roots.loc[row,"Root_Words"]
    unique_id = row
    root_data.append((row, arabic_text))

# Prep Data for the ArabicWord Table
arabic_df = df.drop(['ID', 'Transliteration'], axis=1)
arabic_df = arabic_df.drop_duplicates()
arabic_df["primary_key"] = list(range(2000,19623)) 

# Merge the df to get the RootWords PK (called "index")
arabic_words_df = arabic_df.merge(df_roots, how='inner', left_on='Root_Letters', right_on='Root_Words')
arabic_words_df = arabic_words_df.drop(['Root_Letters', 'Root_Words'], axis=1)

# Rename the index column to RootID to make it clear
arabic_words_df.rename(columns = {'index':'RootID'}, inplace = True)

# Put the dataframe into the format for postgres sql insertion 
arabic_words_data = []
for index, row in arabic_words_df.iterrows():
    arabic_words_data.append( (row['primary_key'], row['ARABIC'], row['RootID']) )

In [35]:
arabic_words_df

,ARABIC,primary_key,RootID
0,بِسْمِ,2000,890
1,السَّمَاءِ,2158,890
2,وَالسَّمَاءَ,2199,890
3,سَمَاوَاتٍ,2302,890
4,الْأَسْمَاءَ,2325,890
...,...,...,...
17618,مَسَدٍ,19606,1727
17619,الصَّمَدُ,19607,118
17620,كُفُوًا,19610,1641
17621,وَقَبَ,19614,1172


In [36]:
df_roots

,index,Root_Words
0,0,ق ل ع
1,1,ط ي ر
2,2,ق ر ح
3,3,ف ت ا
4,4,ر ع د
...,...,...
1794,1794,ح د د
1795,1795,م ح ن
1796,1796,ل د د
1797,1797,ك ب ب


In [ ]:
# Close the DB connection 
    cursor.close()
    connection.close()
    print("PostgreSQL connection is closed")